In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import random

# 设定随机种子
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # 如果使用多个 GPU
    torch.backends.cudnn.deterministic = True  # 保证卷积等操作的确定性
    torch.backends.cudnn.benchmark = False  # 禁用某些优化，保证结果稳定

# 设置随机种子
set_seed(42)


# ---------------------------
# 神经网络定义
# ---------------------------
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size=[50, 50, 50], activation='tanh'):
        super(NeuralNetwork, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, hidden_size[0]))
        if activation == 'tanh':
            layers.append(nn.Tanh())
        elif activation == 'swish':
            layers.append(nn.SiLU())  # Swish 激活函数
        for i in range(len(hidden_size)-1):
            layers.append(nn.Linear(hidden_size[i], hidden_size[i+1]))
            layers.append(nn.Tanh())
        layers.append(nn.Linear(hidden_size[-1], 2))  # 输出 A(x) 和 Φ(x)
        self.network = nn.Sequential(*layers)

        # Xavier初始化
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.network(x)

# ---------------------------
# 自动微分工具
# ---------------------------
def auto_grad(u, x, order=1, allow_unused=False):
    if order == 1:
        grad_outputs = torch.ones_like(u, requires_grad=True)
        return torch.autograd.grad(
            u, x, grad_outputs=grad_outputs,
            retain_graph=True, create_graph=True, allow_unused=allow_unused
        )[0]
    else:
        return auto_grad(auto_grad(u, x, allow_unused=True), x, order=order-1, allow_unused=allow_unused)

# ---------------------------
# 解析解定义
# ---------------------------
def T_anal(x, t, T0, A, W, k, omega, phi):
    """ 温度场的解析解公式 """
    term1 = -W/(2*k)*x
    sqrt_term = np.sqrt(W**4 + 16*k**2*omega**2)
    term2 = (np.sqrt(2)/(4*k)) * np.sqrt(W**2 + sqrt_term) * x
    exp_factor = np.exp(term1 - term2)

    phase = omega*t + phi
    spatial_term = (np.sqrt(2)*omega)/np.sqrt(W**2 + sqrt_term) * x
    sine_factor = np.sin(phase - spatial_term)

    return T0 + A * exp_factor * sine_factor

# ---------------------------
# PINNs正演模型
# ---------------------------
class PINNsForward:
    def __init__(self, layers, k, W, T0, omega, phi0, activation='tanh', device='cpu', lr=0.001):
        self.device = device
        self.dnn = NeuralNetwork(layers[0], layers[1:-1], activation).to(device)
        self.k = torch.tensor(k, dtype=torch.float32, device=device)
        self.W = torch.tensor(W, dtype=torch.float32, device=device)
        self.T0 = torch.tensor(T0, dtype=torch.float32, device=device)
        self.omega = torch.tensor(omega, dtype=torch.float32, device=device)
        self.phi0 = torch.tensor(phi0, dtype=torch.float32, device=device)

        # 优化器配置
        self.optimizer = optim.Adam(self.dnn.parameters(), lr=lr)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=1000, gamma=0.9)

        # 损失记录
        self.loss_history = []
        self.pde_loss_history = []
        self.bc1_loss_history = []  # 边界条件 1 (x=0)
        self.bc2_loss_history = []  # 边界条件 2 (x=0.2)
        self.ic_loss_history = []

    def train(self, epochs, x_train, t_train, S1_bc, T_ic):
        """ 训练过程 """
        # 使用 meshgrid 生成网格数据
        X, T = np.meshgrid(x_train, t_train)
        X_flat = X.reshape(-1, 1)
        T_flat = T.reshape(-1, 1)

        # 转换数据为Tensor并移动到GPU
        x_tensor = torch.tensor(X_flat, requires_grad=True, dtype=torch.float32, device=self.device)
        t_tensor = torch.tensor(T_flat, requires_grad=True, dtype=torch.float32, device=self.device)
        S1_tensor = torch.tensor(S1_bc, dtype=torch.float32, device=self.device)
        T_ic_tensor = torch.tensor(T_ic, dtype=torch.float32, device=self.device)

        # 训练循环
        for epoch in range(epochs):
            self.optimizer.zero_grad()

            # 神经网络预测 A(x) 和 Φ(x)
            outputs = self.dnn(x_tensor)
            A = outputs[:, 0].unsqueeze(1)  # A(x), 形状 (N, 1)
            A = torch.sigmoid(A)  # 使用 Sigmoid 函数将 A 转换到 (0, 1)

            Phi = outputs[:, 1].unsqueeze(1)  # Φ(x), 形状 (N, 1)
            Phi = (Phi + torch.pi) % (2 * torch.pi) - torch.pi  # 变换到 [-π, π] 区间

            # 生成解 T(x,t)
            phase = self.omega * t_tensor + Phi + self.phi0  # 形状 (N, 1)
            T = self.T0 + A * torch.sin(phase)  # 形状 (N, 1)

            # ========== 损失计算 ==========
            # 1. PDE残差
            T_t = auto_grad(T, t_tensor)  # ∂T/∂t
            T_x = auto_grad(T, x_tensor)  # ∂T/∂x
            T_xx = auto_grad(T_x, x_tensor)  # ∂²T/∂x²

            pde_res = T_t - self.k * T_xx - self.W * T_x
            pde_loss = torch.mean(pde_res**2)

            # 2. 边界条件 1 (x=0处)
            bc1_mask = (x_tensor == 0).squeeze()
            if bc1_mask.any():
                bc1_loss = torch.mean((T[bc1_mask] - S1_tensor)**2)
            else:
                bc1_loss = torch.tensor(0.0, device=self.device)

            # 4. 初始条件 (t=0处)
            ic_mask = (t_tensor == 0).squeeze()
            if ic_mask.any():
                ic_loss = torch.mean((T[ic_mask] - T_ic_tensor)**2)
            else:
                ic_loss = torch.tensor(0.0, device=self.device)

            # 动态调整权重
            if epoch < 1000:  # 训练初期，重点优化初始条件
                total_loss = 1e4 * ic_loss
            elif epoch < 5000:  # 逐步加入 PDE 和边界条件
                total_loss = 1e9 * pde_loss + 1e3 * bc1_loss  + 1e3 * ic_loss
            else:  # 训练后期，平衡所有损失
                total_loss = 1e9 * pde_loss + 1e3 * bc1_loss  + 1e3 * ic_loss

            # ========== 反向传播 ==========
            total_loss.backward()
            self.optimizer.step()
            self.scheduler.step()

            # 记录损失
            self.loss_history.append(total_loss.item())
            self.pde_loss_history.append(pde_loss.item())
            self.bc1_loss_history.append(bc1_loss.item())
            self.ic_loss_history.append(ic_loss.item())

            if epoch % 100 == 0:
                print(f"Epoch {epoch:5d} | Total Loss: {total_loss.item():.3e} | "
                      f"PDE: {pde_loss.item():.3e} | BC1: {bc1_loss.item():.3e} | "
                      f"IC: {ic_loss.item():.3e}")

    def predict(self, x):
        """ 预测 A(x) 和 Φ(x) """
        self.dnn.eval()
        with torch.no_grad():
            x_tensor = torch.tensor(x, dtype=torch.float32, device=self.device)
            outputs = self.dnn(x_tensor)
            A = outputs[:, 0].cpu().numpy()
            Phi = outputs[:, 1].cpu().numpy()
            return A, Phi

# ---------------------------
# 参数配置
# ---------------------------
# 物理参数
k = 1.54e-6    # 扩散系数 (m²/s)
W = 2.38e-6    # 对流系数 (m/s)
T0 = 0.0       # 基础温度
A_anal = 1.0   # 解析解振幅
omega = 2*np.pi/(24*3600)  # 角频率 (rad/s)
phi0 = 0.0     # 初始相位

# 空间和时间参数
x_min, x_max = 0.0, 0.2   # 空间范围 (m)
t_min, t_max = 0.0, 0.5*24*3600  # 时间范围 (s)
num_x = 200              # 空间采样点数
num_t = 1000               # 时间采样点数

# 训练参数
hidden_layers = [2, 200, 200, 200, 200, 2]  # 神经网络结构
epochs = 5000                # 训练轮数
lr = 1e-3                     # 初始学习率

# ---------------------------
# 数据生成
# ---------------------------
# 生成训练数据
x_vals = np.linspace(x_min, x_max, num_x).reshape(-1, 1)
t_vals = np.linspace(t_min, t_max, num_t).reshape(-1, 1)

# 使用解析解生成边界条件 S1(t)（x=0处的温度）
S1 = T_anal(0, t_vals, T0, A_anal, W, k, omega, phi0)

# 使用解析解生成初始条件 T(x, t=0)
T_ic = T_anal(x_vals, 0, T0, A_anal, W, k, omega, phi0)

# 检查是否有可用的 GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# ---------------------------
# 模型训练
# ---------------------------
# 初始化模型
model = PINNsForward(layers=[1]+hidden_layers+[2],  # 输入为 x，输出为 A 和 Φ
                     k=k, W=W, T0=T0, omega=omega, phi0=phi0,
                     activation='tanh',
                     device=device,
                     lr=lr)

# 开始训练
print("Start training...")
model.train(epochs, x_vals, t_vals, S1, T_ic)

Using device: cuda
Start training...
Epoch     0 | Total Loss: 5.686e+02 | PDE: 6.576e-10 | BC1: 1.249e-01 | IC: 5.686e-02
Epoch   100 | Total Loss: 3.887e-01 | PDE: 6.635e-10 | BC1: 1.127e-01 | IC: 3.887e-05
Epoch   200 | Total Loss: 2.408e-01 | PDE: 5.485e-10 | BC1: 1.018e-01 | IC: 2.408e-05
Epoch   300 | Total Loss: 1.246e-01 | PDE: 4.530e-10 | BC1: 8.892e-02 | IC: 1.246e-05
Epoch   400 | Total Loss: 5.425e-02 | PDE: 3.953e-10 | BC1: 7.707e-02 | IC: 5.425e-06
Epoch   500 | Total Loss: 2.585e-02 | PDE: 3.751e-10 | BC1: 6.883e-02 | IC: 2.585e-06
Epoch   600 | Total Loss: 1.928e-02 | PDE: 3.727e-10 | BC1: 6.472e-02 | IC: 1.928e-06
Epoch   700 | Total Loss: 1.843e-02 | PDE: 3.733e-10 | BC1: 6.323e-02 | IC: 1.843e-06
Epoch   800 | Total Loss: 1.834e-02 | PDE: 3.735e-10 | BC1: 6.280e-02 | IC: 1.834e-06
Epoch   900 | Total Loss: 3.511e-02 | PDE: 3.801e-10 | BC1: 6.234e-02 | IC: 3.511e-06
Epoch  1000 | Total Loss: 6.420e+01 | PDE: 3.724e-10 | BC1: 6.383e-02 | IC: 1.862e-06
Epoch  1100 | Tot

In [ ]:

# ---------------------------
# 结果可视化
# ---------------------------
# 训练损失曲线
plt.figure(figsize=(8, 6))
plt.semilogy(model.loss_history, label='Total Loss')
plt.semilogy(model.pde_loss_history, '--', label='PDE Loss')
plt.semilogy(model.bc1_loss_history, '--', label='BC Loss')
plt.semilogy(model.ic_loss_history, '--', label='IC Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (log scale)')
plt.title('Training Loss History')
plt.legend()
plt.grid(True)
plt.show()

# 边界条件验证（x=0）
x_bc = np.zeros((num_t, 1))  # x=0处
A_pred, Phi_pred = model.predict(x_bc)
A_pred = torch.tensor(A_pred)
A_pred = torch.sigmoid(A_pred)
Phi_pred = (Phi_pred + torch.pi) % (2 * torch.pi) - torch.pi  # 变换到 [-π, π] 区间

T_pred_bc = T0 + A_pred * np.sin(omega * t_vals + Phi_pred + phi0)
T_pred_bc = T_pred_bc[:,1]
plt.figure(figsize=(8, 6))
plt.plot(t_vals/3600, S1, 'r--', label='True S1(t)')
plt.plot(t_vals/3600, T_pred_bc, 'b-', label='Predicted T(0,t)')
plt.xlabel('Time (hours)')
plt.ylabel('Temperature')
plt.title('Boundary Condition Validation at x=0')
plt.legend()
plt.grid(True)
plt.show()

# 对比解析解和神经网络预测的解（x=0.2）
x_02 = 0.2 * np.ones_like(t_vals)  # x=0.2处，形状 (200, 1)
T_anal_02 = T_anal(x_02, t_vals, T0, A_anal, W, k, omega, phi0)  # 解析解，形状 (200, 1)

# 神经网络预测的 A 和 Φ
A_pred_02, Phi_pred_02 = model.predict(x_02)  # A_pred_02 和 Phi_pred_02 的形状应为 (200, 1)
A_pred_02 = torch.tensor(A_pred_02)
A_pred_02 = torch.sigmoid(A_pred_02)
Phi_pred_02 = (Phi_pred_02 + torch.pi) % (2 * torch.pi) - torch.pi  # 变换到 [-π, π] 区间

# 确保 A_pred_02 和 Phi_pred_02 的形状正确
A_pred_02 = A_pred_02.reshape(-1, 1)  # 形状 (200, 1)
Phi_pred_02 = Phi_pred_02.reshape(-1, 1)  # 形状 (200, 1)

# 生成神经网络预测的解
T_pred_02 = T0 + A_pred_02 * np.sin(omega * t_vals + Phi_pred_02 + phi0)  # 形状 (200, 1)

# 绘制对比图
plt.figure(figsize=(8, 6))
plt.plot(t_vals/3600, T_anal_02, 'r--', label='Analytical T(0.2,t)')
plt.plot(t_vals/3600, T_pred_02, 'b-', label='Predicted T(0.2,t)')
plt.xlabel('Time (hours)')
plt.ylabel('Temperature')
plt.title('Comparison of Analytical and PINNs Solution at x=0.2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 对比解析解和神经网络预测的解（x=0.2）
x_01 = 0.1 * np.ones_like(t_vals)  # x=0.2处，形状 (200, 1)
T_anal_01 = T_anal(x_01, t_vals, T0, A_anal, W, k, omega, phi0)  # 解析解，形状 (200, 1)

# 神经网络预测的 A 和 Φ
A_pred_01, Phi_pred_01 = model.predict(x_01)  # A_pred_02 和 Phi_pred_02 的形状应为 (200, 1)
A_pred_01 = torch.tensor(A_pred_01)
A_pred_01 = torch.sigmoid(A_pred_01)
Phi_pred_01 = (Phi_pred_01 + torch.pi) % (2 * torch.pi) - torch.pi  # 变换到 [-π, π] 区间

# 确保 A_pred_02 和 Phi_pred_02 的形状正确
A_pred_01 = A_pred_01.reshape(-1, 1)  # 形状 (200, 1)
Phi_pred_01 = Phi_pred_01.reshape(-1, 1)  # 形状 (200, 1)

# 生成神经网络预测的解
T_pred_01 = T0 + A_pred_01 * np.sin(omega * t_vals + Phi_pred_01 + phi0)  # 形状 (200, 1)

# 绘制对比图
plt.figure(figsize=(8, 6))
plt.plot(t_vals/3600, T_anal_01, 'r--', label='Analytical T(0.2,t)')
plt.plot(t_vals/3600, T_pred_01, 'b-', label='Predicted T(0.2,t)')
plt.xlabel('Time (hours)')
plt.ylabel('Temperature')
plt.title('Comparison of Analytical and PINNs Solution at x=0.2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 绘制对比图
plt.figure(figsize=(8, 6))
plt.plot(t_vals/3600, T_pred_bc, 'r--', label='Predicted T(0 m,t)')
plt.plot(t_vals/3600, T_pred_02, 'b-', label='Predicted T(0.2 m,t)')
plt.plot(t_vals/3600, T_pred_01, 'b-', label='Predicted T(0.1 m,t)')
plt.xlabel('Time (hours)')
plt.ylabel('Temperature')
plt.title('Comparison of Analytical and PINNs Solution at x=0.2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ---------------------------
# 初始条件（IC）对比
# ---------------------------
# 在 t=0 时计算解析解和神经网络预测的解
t_ic = 0.0  # 初始时间
x_ic = np.linspace(x_min, x_max, num_x).reshape(-1, 1)  # 空间坐标

# 解析解在 t=0 时的值
T_anal_ic = T_anal(x_ic, t_ic, T0, A_anal, W, k, omega, phi0)

# 神经网络预测的 A 和 Φ
A_pred_ic, Phi_pred_ic = model.predict(x_ic)
A_pred_ic = torch.tensor(A_pred_ic)
A_pred_ic = torch.sigmoid(A_pred_ic)
Phi_pred_ic = (Phi_pred_ic + torch.pi) % (2 * torch.pi) - torch.pi  # 变换到 [-π, π] 区间

# 生成神经网络预测的解
T_pred_ic = T0 + A_pred_ic * np.sin(omega * t_ic + Phi_pred_ic + phi0)

# 绘制初始条件对比图
plt.figure(figsize=(8, 6))
plt.plot(x_ic, T_anal_ic, 'r--', label='Analytical T(x, t=0)')
plt.plot(x_ic, T_pred_ic, 'b-', label='Predicted T(x, t=0)')
plt.xlabel('Position x (m)')
plt.ylabel('Temperature')
plt.title('Comparison of Analytical and PINNs Solution at t=0')
plt.legend()
plt.grid(True)
plt.show()